### Final Project Requirements/notes: https://docs.google.com/document/d/1mwYbYJHkB7kpx4tNflKh54jN9_oOscw3p4k5fsmn3bc/edit

### Link with all Data: https://www.sec.gov/dera/data/financial-statement-and-notes-data-set.html
- using NUM file only for now (data set of all numeric XBRL facts presented on the primary financial statements)

# 1. Import and clean DF

In [1]:
import pandas as pd
q414numbers = pd.read_table('2014q4_notes/num.tsv', encoding ='latin1')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### From "Financial Statement and Notes Data Sets" Readme:

**adsh - EDGAR accession number**: a unique identifier assigned automatically to an accepted submission by the EDGAR Filer System; The first set of numbers (0001193125) is the CIK of the entity submitting the filing. The next 2 numbers (18) represent the year. The last series of numbers represent a sequential count of submitted filings from that CIK. The count is usually, but not always, reset to 0 at the start of each calendar year.
- **TODO**: separate these numbers to identify a company or a financial filing, there were 6,492 individual filings

#### A lot of null values for footnotes and coregistrants (majority of rows); will remove these columns for now

In [2]:
q414numbers = q414numbers.drop(columns=['footnote','coreg'])
q414numbers = q414numbers.dropna()

In [3]:
# q414numbers.isnull().sum()

In [4]:
# q414numbers.describe(include='all') #still have 5million+ data points

In [5]:
# len(set(q414numbers.entity_CIK))

# 2. Group by filing # (adsh) and filter just for values

In [40]:
adsh_grouped = q414numbers[['adsh','value']].groupby('adsh')
# adsh_grouped.groups


In [41]:
len(adsh_grouped.groups) #check for the total number of filings in this quarter

7655

# 3. Get rid of negative values
## QUESTION: HOW SHOULD I DEAL WITH DECIMALS WITH BENFORD? POTENTIALLY MULTIPLY by 100?

In [45]:
#get rid of any negative values and any duplicates within the list of numbers
def no_negs(number):
    return list(set(number.abs()))


In [46]:
grouped_abs_val = adsh_grouped.agg(no_negs)

In [47]:
grouped_abs_val.head()#value is now a list of positive unique numbers, split up by filer and that specific filing

,value
adsh,
0000002178-14-000064,"[0.0, 960000.0, 1.7, 3.13, 1.72, 5.1, 5.2, 1.0..."
0000003146-14-000006,"[0.0, 1.0, 51200000.0, 2.0, 4.0, 5.0, 6400000...."
0000003146-14-000009,"[0.0, 51200000.0, 1.0, 2.0, 4.0, 5.0, 6400000...."
0000003499-14-000018,"[0.0, 1.0, 2.71, 3.46, 2.75, 3.25, 13824000.0,..."
0000003570-14-000268,"[0.0, 151808000.0, 480000000.0, 1.71, 4.0, 0.0..."


In [48]:
grouped_abs_val.value[0:10]

adsh
0000002178-14-000064    [0.0, 960000.0, 1.7, 3.13, 1.72, 5.1, 5.2, 1.0...
0000003146-14-000006    [0.0, 1.0, 51200000.0, 2.0, 4.0, 5.0, 6400000....
0000003146-14-000009    [0.0, 51200000.0, 1.0, 2.0, 4.0, 5.0, 6400000....
0000003499-14-000018    [0.0, 1.0, 2.71, 3.46, 2.75, 3.25, 13824000.0,...
0000003570-14-000268    [0.0, 151808000.0, 480000000.0, 1.71, 4.0, 0.0...
0000004127-14-000046    [6400000.0, 0.0, 2.38, 1.33, 0.25, 1.0, 256000...
0000004187-14-000043    [0.0, 44032000.0, 0.5, 2.0, 2304000.0, 1.0, 4....
0000004457-14-000051    [0.0, 0.25, 3.0, 7.98, 10453000.0, 5141717000....
0000004904-14-000097    [2432000000.0, 0.0, 0.5, 1.5, 1.0, 3.5, 6.5, 4...
0000004977-14-000138    [0.0, 1.5, 1.0, 2.0, 4.93, 5.31, 5.0, 25600000...
Name: value, dtype: object

In [17]:
# type(grouped_abs_val.value)

pandas.core.series.Series

In [18]:
# test = grouped_abs_val.value[0]
# print(len(test),test)

In [18]:
# type(test)

list

In [19]:
# for number in test:
#     benford = int(str(number)[0])
#     print(benford)

In [19]:
# for val in grouped_abs_val.value:
#     print(val)

In [20]:
len(grouped_abs_val.value)


7655

# 4. Get first digit from each value number

In [49]:
def first_digit(number): 
    return int(str(number)[0]) #convert to string first and then extract first index position

In [50]:
# all_nums = [v for v in grouped_abs_val.value[0]]
# all_nums

In [51]:
for v in grouped_abs_val.value:
    first_nums = [first_digit(num) for num in v]

In [52]:
print(len(grouped_abs_val.value[20]), grouped_abs_val.value[20])

374 [64000000.0, 2464000000.0, 2.0, 0.0, 3.0, 32000000.0, 240000000000.0, 1088000000.0, 128000000.0, 4000000000.0, 4.24, 1.0, 32.07, 35.7, 36.27, 35.39, 38.42, 37.55, 40.44, 37.66, 37.94, 36.62, 45.81, 46.55, 351014974.0, 409000000.0, 25000000.0, 9177000000.0, 1913000000.0, 2169000000.0, 57000000.0, 121000000.0, 6265000000.0, 0.0164, 729000000.0, 350517337.0, 91.56, 237662.0, 95.56, 376000000.0, 50000000.0, 82000000.0, 690000000.0, 242000000.0, 114000000.0, 146000000.0, 2450000000.0, 18000000.0, 1657000.0, 334000.0, 235000000.0, 203000000.0, 75000000.0, 139000000.0, 11000000.0, 299000000.0, 2155000000.0, 107000000.0, 651000000.0, 907000000.0, 612000000.0, 644000000.0, 1668000000.0, 932000000.0, 1988000000.0, 4000000.0, 868000000.0, 100000000.0, 3076000000.0, 68000000.0, 117000.0, 442678.0, 765000000.0, 1021000000.0, 29000000.0, 93000000.0, 2653000000.0, 349000000.0, 928000000.0, 957000000.0, 189000000.0, 1533000000.0, 1740000000.0, 1531000000.0, 707000000.0, 1494000000.0, 246000000.0, 

In [53]:
def frequencies(first_digits):
    counts = [0]*10
    for x in first_digits:
        if x > 0:
            counts[x] += 1 
    total = sum(counts)
    freq = [count/total for count in counts]
    print(sum(freq)) #frequencies should sum up to 1
    return freq[1:] #same as going from element 1 through the end (i.e. 9)

In [55]:
frequencies(first_nums)

1.0


[0.26791808873720135,
 0.22525597269624573,
 0.12457337883959044,
 0.1075085324232082,
 0.08873720136518772,
 0.052901023890784986,
 0.042662116040955635,
 0.042662116040955635,
 0.04778156996587031]

In [ ]:
# def frequencies(first_digits):
#     counts = [0]*10
#     for x in first_digits:
#         counts[x] += 1 
#     total = sum(counts)
#     freq = [count/total for count in counts]
#     print(sum(freq)) #frequencies should sum up to 1
#     return freq[1:] #same as going from element 1 through the end (i.e. 9)

In [165]:
# freq = [0] * 10
import math
def kl_divergence(freq):
    kl_div = 0.0
    for d in range(1, 10):
        Q = (math.log(d+1) - math.log(d)) / math.log(10) #calculates theoretical benfords (perfect benny freqs)
        P = freq[d-1]
        kl_div += math.log(P / Q) * P
#         print((freq))

    return int(kl_div)

In [166]:
kl_divergence(benfords_law)

0

In [127]:

r = range(10)
benfords_law = [math.log10(1 + 1/digit) for digit in r if digit != 0]
benfords_law

[0.3010299956639812,
 0.17609125905568124,
 0.12493873660829993,
 0.09691001300805642,
 0.07918124604762482,
 0.06694678963061322,
 0.05799194697768673,
 0.05115252244738129,
 0.04575749056067514]

In [128]:
sum(benfords_law) 

1.0

In [171]:
fds[0][1]

TypeError: 'int' object is not subscriptable

In [168]:
kl_divergence(fds[0])

TypeError: 'int' object is not subscriptable

In [ ]:
# 2. Break out adsh to cik and filing number (if applicable)

In [ ]:

# s = q414numbers['adsh'].str.split('-', n = 1, expand = True)
# q414numbers['entity_CIK'] = s[0]
# q414numbers['filing_number'] = s[1]
# q414numbers.head()

# System; The first set of numbers (0001193125) is the CIK of the entity submitting the filing. 
# The next 2 numbers (18) represent the year. 
# The last series of numbers represent a sequential count of submitted filings from that CIK. 
# The count is usually, but not always, reset to 0 at the start of each calendar year.

In [ ]:
# 3. come back to this: need to see if this single company had multiple filings

In [ ]:
# len(q414numbers.entity_CIK == '0001171843')